In [5]:
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau

gaussian_model = load_model('../models/GaussianBlur/gaussian_model.h5')
gaussian_model._name = "gaussian_model"

laplacian_model = load_model('../models/Laplacian/laplacian_model.h5')
laplacian_model._name = "laplacian_model"

input_gaussian = Input(shape=(224, 224, 3), name="input_gaussian")
input_laplacian = Input(shape=(224, 224, 3), name="input_laplacian")

output_gaussian = gaussian_model(input_gaussian)
output_laplacian = laplacian_model(input_laplacian)

merged = Concatenate()([output_gaussian, output_laplacian])
final_output = Dense(4, activation='softmax', name="classification")(merged)

fusion_model = Model(inputs=[input_gaussian, input_laplacian], outputs=final_output)

optimizer = SGD(learning_rate=0.01, momentum=0.9)
fusion_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.8,       # Reduce en 20%
    patience=5,       # Cada 5 épocas sin mejora
    verbose=1
)

In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import pandas as pd
import os

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
KERNEL_SIZE = 15

csv_path = "../dataset/HandInfo.csv"
image_root = "../dataset/image"

# === Leer CSV y procesar etiquetas ===
df = pd.read_csv(csv_path)

# Mapear aspectOfHand a etiquetas numéricas
class_names = sorted(df["aspectOfHand"].unique())
class_to_idx = {name: i for i, name in enumerate(class_names)}
df["label"] = df["aspectOfHand"].map(class_to_idx)

def z_score_normalization(image):
    mean = image.mean()
    std = image.std()
    return (image - mean) / std

def preprocess_gaussian(image_path):
    image_path = image_path.decode("utf-8")
    image = cv2.imread(image_path)

    if image is None:
        print(f"Error loading image: {image_path}")
        return np.zeros((*IMG_SIZE, 3), dtype=np.float32), -1  # Evitar errores

    image = cv2.resize(image, IMG_SIZE).astype(np.float32)
    low_freq_image = cv2.GaussianBlur(image, (KERNEL_SIZE, KERNEL_SIZE), 0)
    normalized_image = z_score_normalization(low_freq_image)

    return normalized_image

def preprocess_laplacian(image_path):
    if isinstance(image_path, bytes):  # A veces llega como bytes
        image_path = image_path.decode('utf-8')

    image = cv2.imread(image_path)
    if image is None:
        print(f"Error loading image: {image_path}")
        return np.zeros((224, 224, 3), dtype=np.float32)  # Evitar errores posteriores

    image = cv2.resize(image, IMG_SIZE).astype(np.float32)
    laplacian = cv2.Laplacian(image, cv2.CV_32F)
    high_freq_image = z_score_normalization(laplacian)
    return high_freq_image

def get_tf_preprocess(preprocess_fn):
    def tf_preprocess(image_path, label):
        image = tf.numpy_function(preprocess_fn, [image_path], tf.float32)
        image.set_shape((*IMG_SIZE, 3))
        label.set_shape(())
        return image, label
    return tf_preprocess

def get_labels(split_name):
    folder_path = os.path.join(image_root, split_name)
    available_images = set(os.listdir(folder_path))

    df_split = df.copy()
    df_split = df_split[df_split["imageName"].isin(available_images)]

    labels = tf.constant(df_split["label"].values)
    return labels


def create_dataset_from_split(split_name, preprocess_fn):
    folder_path = os.path.join(image_root, split_name)
    available_images = set(name for name in os.listdir(folder_path))
    df_split = df.copy()
    
    split_df = df_split[df_split["imageName"].isin(available_images)]
    split_df = split_df.sort_values("imageName")

    image_paths = [os.path.join(folder_path, name) for name in split_df["imageName"]]
    image_paths = tf.constant(image_paths)
    labels = tf.constant(split_df["label"].values)

    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.map(preprocess_fn, num_parallel_calls=tf.data.AUTOTUNE)

    return dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Dataset con preprocesamiento Gaussian
gaussian_ds = create_dataset_from_split("train", get_tf_preprocess(preprocess_gaussian))
gaussian_val = create_dataset_from_split("val", get_tf_preprocess(preprocess_gaussian))

# Dataset con preprocesamiento Laplacian
laplacian_ds = create_dataset_from_split("train", get_tf_preprocess(preprocess_laplacian))
laplacian_val = create_dataset_from_split("val", get_tf_preprocess(preprocess_laplacian))

def format_fusion_dataset(gaussian_ds, laplacian_ds):
    return tf.data.Dataset.zip((gaussian_ds, laplacian_ds)).map(
        lambda g, l: ((g[0], l[0]), g[1]),  # inputs: (gaussian_img, laplacian_img), label: g[1]
        num_parallel_calls=tf.data.AUTOTUNE
    )

train_ds_fusion = format_fusion_dataset(gaussian_ds, laplacian_ds).shuffle(1000)
val_ds_fusion = format_fusion_dataset(gaussian_val, laplacian_val)

2025-04-09 03:41:08.829024: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:67: Filling up shuffle buffer (this may take a while): 54 of 1000


In [4]:
from tensorflow.keras.optimizers.legacy import SGD

fusion_model.compile(
    optimizer= SGD(learning_rate=0.09, momentum=0.9, decay=0.001),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

fusion_model.fit(
    train_ds_fusion,
    epochs=5,
    validation_data=val_ds_fusion,
    callbacks=[lr_scheduler]
)

Epoch 1/5


2025-04-09 03:39:18.871145: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:42: Filling up shuffle buffer (this may take a while): 56 of 1000
2025-04-09 03:39:38.847454: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:42: Filling up shuffle buffer (this may take a while): 146 of 1000
2025-04-09 03:39:58.769104: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:42: Filling up shuffle buffer (this may take a while): 235 of 1000
2025-04-09 03:40:00.186049: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.
2025-04-09 03:40:00.186108: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 38535424 bytes after encountering the first element of size 38535424 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size
2025-04-09 03:40:01.278419: W tensorflow

InvalidArgumentError: Graph execution error:

Detected at node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/ale/Documentos/PID/genero_manos/venv/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/ale/Documentos/PID/genero_manos/venv/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/ale/Documentos/PID/genero_manos/venv/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/ale/Documentos/PID/genero_manos/venv/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line